# 一、加载常用库

In [2]:
import os
import matplotlib.pyplot as plt

%matplotlib inline

import warnings #不显示乱七八糟的warning
warnings.filterwarnings("ignore")

In [3]:
from tianchi import *
%load_ext autoreload
%autoreload 2

# 二、设置环境

## 2.1 数据和csv的位置，根据需要修改

In [4]:
luna_path = '/Volumes/solo/Luna16/'
luna_subset_path = luna_path + 'subset0/'
file_list = glob(luna_subset_path + "*.mhd")
df_node = pd.read_csv(luna_path + 'CSVFILES/' + 'annotations.csv')
npy_list=glob(luna_path + 'npy/' + "images_*.npy")

## 2.2 临时文件和输出文件的位置

In [5]:
output_path = luna_path + 'npy/'
working_path = luna_path + 'output/'

## 2.3 检查下文件夹，如果没有的话新建

In [6]:
if os.path.isdir(luna_path + '/npy'):
    pass
else:
    os.mkdir(luna_path + '/npy')
    
if os.path.isdir(luna_path + '/output'):
    pass
else:
    os.mkdir(luna_path + '/output')

In [7]:
df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(file_list, file_name))
df_node = df_node.dropna()
df_node.head(5)

,seriesuid,coordX,coordY,coordZ,diameter_mm,file
23,1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896...,-100.567944,67.260517,-231.816619,6.440879,/Volumes/solo/Luna16/subset0/1.3.6.1.4.1.14519...
25,1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...,46.188539,48.402806,-108.578632,13.596471,/Volumes/solo/Luna16/subset0/1.3.6.1.4.1.14519...
26,1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524...,36.392044,76.771663,-123.321911,4.343200,/Volumes/solo/Luna16/subset0/1.3.6.1.4.1.14519...
28,1.3.6.1.4.1.14519.5.2.1.6279.6001.111172165674...,136.434059,117.765579,-181.947817,4.681382,/Volumes/solo/Luna16/subset0/1.3.6.1.4.1.14519...
86,1.3.6.1.4.1.14519.5.2.1.6279.6001.124154461048...,145.967465,-161.197634,-312.071347,6.378436,/Volumes/solo/Luna16/subset0/1.3.6.1.4.1.14519...


In [8]:
print ('数据条数：%d' % df_node.shape[0])

数据条数：112


In [7]:
tic = time.time()
loaddata(file_list,df_node,output_path)
toc = time.time()
print(u'用时:%0.2f秒' % (toc - tic))

100%|██████████| 89/89 [01:34<00:00,  1.18it/s]

用时:95.02秒


In [8]:
tic = time.time()
save_mask(npy_list)
toc = time.time()
print(u'用时:%0.2f秒' % (toc - tic))

100%|██████████| 112/112 [01:15<00:00,  1.64it/s]

肺部掩模已生成！
用时:75.64秒


In [9]:
tic = time.time()
save_train_test(luna_path,working_path)
toc = time.time()
print(u'用时:%0.2f秒' % (toc - tic))

100%|██████████| 112/112 [00:16<00:00,  6.48it/s]


(1/2) 肺部样本图片和结节掩模已生成！
(2/2) train和test数据已经保存！

	训练数据集： /Volumes/solo/Luna16/output/trainImages.npy	 训练样本： 269
	训练掩模集： /Volumes/solo/Luna16/output/trainMasks.npy	 样本占比： 80 %

	测试数据集： /Volumes/solo/Luna16/output/testImages.npy	 测试样本： 67
	测试掩模集： /Volumes/solo/Luna16/output/testMasks.npy	 样本占比： 19 %


In [9]:
import numpy as np
from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K



K.set_image_dim_ordering('th')  # Theano dimension ordering in this code

img_rows = 512
img_cols = 512

smooth = 1.


def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_np(y_true,y_pred):
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)


def get_unet():
    inputs = Input((1,img_rows, img_cols))
    conv1 = Convolution2D(32, (3, 3), padding="same", activation="relu")(inputs)
    conv1 = Convolution2D(32, (3, 3), padding="same", activation="relu")(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(64, (3, 3), padding="same", activation="relu")(pool1)
    conv2 = Convolution2D(64, (3, 3), padding="same", activation="relu")(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(128, (3, 3), padding="same", activation="relu")(pool2)
    conv3 = Convolution2D(128, (3, 3), padding="same", activation="relu")(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(256, (3, 3), padding="same", activation="relu")(pool3)
    conv4 = Convolution2D(256, (3, 3), padding="same", activation="relu")(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(512, (3, 3), padding="same", activation="relu")(pool4)
    conv5 = Convolution2D(512, (3, 3), padding="same", activation="relu")(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=1)
    conv6 = Convolution2D(256, (3, 3), padding="same", activation="relu")(up6)
    conv6 = Convolution2D(256, (3, 3), padding="same", activation="relu")(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=1)
    conv7 = Convolution2D(128, (3, 3), padding="same", activation="relu")(up7)
    conv7 = Convolution2D(128, (3, 3), padding="same", activation="relu")(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=1)
    conv8 = Convolution2D(64, (3, 3), padding="same", activation="relu")(up8)
    conv8 = Convolution2D(64, (3, 3), padding="same", activation="relu")(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=1)
    conv9 = Convolution2D(32, (3, 3), padding="same", activation="relu")(up9)
    conv9 = Convolution2D(32, (3, 3), padding="same", activation="relu")(conv9)

    conv10 = Convolution2D(1, (1, 1), activation="sigmoid")(conv9)

    model = Model(input=inputs, output=conv10)

    model.compile(optimizer=Adam(lr=1.0e-5), loss=dice_coef_loss, metrics=[dice_coef])

    return model


def train_and_predict(use_existing):
    print('-'*30)
    print('Loading and preprocessing train data...')
    print('-'*30)
    imgs_train = np.load(working_path+"trainImages.npy").astype(np.float32)
    imgs_mask_train = np.load(working_path+"trainMasks.npy").astype(np.float32)

    imgs_test = np.load(working_path+"testImages.npy").astype(np.float32)
    imgs_mask_test_true = np.load(working_path+"testMasks.npy").astype(np.float32)
    
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization

    imgs_train -= mean  # images should already be standardized, but just in case
    imgs_train /= std

    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    # Saving weights to unet.hdf5 at checkpoints
    model_checkpoint = ModelCheckpoint('unet.hdf5', monitor='loss', save_best_only=True)
    #
    # Should we load existing weights? 
    # Set argument for call to train_and_predict to true at end of script
    if use_existing:
        model.load_weights('./unet.hdf5')
        
    # 
    # The final results for this tutorial were produced using a multi-GPU
    # machine using TitanX's.
    # For a home GPU computation benchmark, on my home set up with a GTX970 
    # I was able to run 20 epochs with a training set size of 320 and 
    # batch size of 2 in about an hour. I started getting reseasonable masks 
    # after about 3 hours of training. 
    #
    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    
    #原代码batch_size=2, nb_epoch=20，我显存不够，所以改cpu跑
    
    model.fit(imgs_train, imgs_mask_train, batch_size=1, nb_epoch=1, verbose=1, shuffle=True,
              callbacks=[model_checkpoint])

    # loading best weights from training session
    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('./unet.hdf5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    num_test = len(imgs_test)
    imgs_mask_test = np.ndarray([num_test,1,512,512],dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([imgs_test[i:i+1]], verbose=0)[0]
    np.save('masksTestPredicted.npy', imgs_mask_test)
    mean = 0.0
    for i in range(num_test):
        mean+=dice_coef_np(imgs_mask_test_true[i,0], imgs_mask_test[i,0])
    mean/=num_test
    print("Mean Dice Coeff : ",mean)
    
if __name__ == '__main__':
    tic = time.time()
    train_and_predict(False)
    toc = time.time()
    print(u'用时:%0.2f秒' % (toc - tic))

Using TensorFlow backend.


"if __name__ == '__main__':\n    tic = time.time()\n    train_and_predict(False)\n    toc = time.time()\n    print(u'\xe7\x94\xa8\xe6\x97\xb6:%0.2f\xe7\xa7\x92' % (toc - tic))"